In [5]:
[;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fashion-vision-apparel-classification-challenge/train.csv
/kaggle/input/fashion-vision-apparel-classification-challenge/test.csv


In [ ]:
pip install torchmetrics

In [6]:
#importing required libraries
import torch 
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import train_test_split
from torchmetrics.classification import Accuracy

In [7]:

# ---------- Load and split data ----------
df = pd.read_csv('/kaggle/input/fashion-vision-apparel-classification-challenge/train.csv')
x = df.drop(columns=['label']).values
y = df['label'].values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# ---------- Custom Dataset Class ----------
class MyData(Dataset):
    def __init__(self, x, y):
        self.x = torch.tensor(x, dtype=torch.float32).view(-1, 1, 28, 28)  # NCHW format
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

# ---------- Create Datasets & Loaders ----------
training_data = MyData(x_train, y_train)
testing_data = MyData(x_test, y_test)

train_loader = DataLoader(training_data, batch_size=32, shuffle=True)
val_loader = DataLoader(testing_data, batch_size=32)


In [9]:

# ----------- Model Definition -----------
class MYCNN(nn.Module):
    def __init__(self):
        super(MYCNN, self).__init__()
        self.Conv_layers = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 32, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 16, kernel_size=(2, 2), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.fc_layer = nn.Sequential(
            nn.Flatten(),
            nn.Linear(14 * 14 * 16, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )

    def forward(self, x):  
        x = self.Conv_layers(x)
        x = self.fc_layer(x)
        return x

# ----------- Initialize -----------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MYCNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 10

# Accuracy metrics
train_acc = Accuracy(task="multiclass", num_classes=10).to(device)
val_acc = Accuracy(task="multiclass", num_classes=10).to(device)

# ----------- Training Loop -----------
for epoch in range(epochs):
    model.train()
    train_loss = 0
    train_acc.reset()

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_acc.update(outputs, labels)

    avg_train_loss = train_loss / len(train_loader)
    train_accuracy = train_acc.compute().item()

    # ----------- Validation Loop -----------
    model.eval()
    val_loss = 0.0
    val_acc.reset()

    with torch.no_grad():
        for val_images, val_labels in val_loader:
            val_images, val_labels = val_images.to(device), val_labels.to(device)
            val_outputs = model(val_images)
            loss = criterion(val_outputs, val_labels)

            val_loss += loss.item()
            val_acc.update(val_outputs, val_labels)

        avg_val_loss = val_loss / len(val_loader)
        val_accuracy = val_acc.compute().item()

    print(f"Epoch {epoch+1} | "
          f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.4f} | "
          f"Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.4f}")


Epoch 1 | Train Loss: 0.5171, Train Acc: 0.8091 | Val Loss: 0.3304, Val Acc: 0.8802
Epoch 2 | Train Loss: 0.3131, Train Acc: 0.8871 | Val Loss: 0.2877, Val Acc: 0.8980
Epoch 3 | Train Loss: 0.2624, Train Acc: 0.9040 | Val Loss: 0.3025, Val Acc: 0.8936
Epoch 4 | Train Loss: 0.2190, Train Acc: 0.9199 | Val Loss: 0.3024, Val Acc: 0.8893
Epoch 5 | Train Loss: 0.1848, Train Acc: 0.9318 | Val Loss: 0.2596, Val Acc: 0.9114
Epoch 6 | Train Loss: 0.1555, Train Acc: 0.9444 | Val Loss: 0.2423, Val Acc: 0.9129
Epoch 7 | Train Loss: 0.1239, Train Acc: 0.9555 | Val Loss: 0.2647, Val Acc: 0.9171
Epoch 8 | Train Loss: 0.1004, Train Acc: 0.9635 | Val Loss: 0.2826, Val Acc: 0.9137
Epoch 9 | Train Loss: 0.0782, Train Acc: 0.9719 | Val Loss: 0.3420, Val Acc: 0.9058
Epoch 10 | Train Loss: 0.0675, Train Acc: 0.9758 | Val Loss: 0.3288, Val Acc: 0.9114


In [15]:
x_test=pd.read_csv('/kaggle/input/fashion-vision-apparel-classification-challenge/test.csv')
ids=x_test['id']
x_test=x_test.drop(columns=['id']).values

class test_data(Dataset):
    def __init__(self,x):
        self.x=torch.tensor(x,dtype=torch.float).view(-1,1,28,28)
    def __len__(self):
        return len(self.x)
    def __getitem__(self,idx):
        return self.x[idx]

test_data=test_data(x_test)
test_loder=DataLoader(test_data,batch_size=32)    
    

In [17]:
def predict(model,loader,device):
    model.eval()
    prediction =[]
    with torch.no_grad():
        for images in loader:
            images=images.to(device)
            output=model(images)
            pred=torch.argmax(output,dim=1)
            prediction.extend(pred.cpu().numpy())
    return prediction
prediction=predict(model,test_loder,device)
result=pd.DataFrame({
    'id':ids,
    'label':prediction
})
result.to_csv('result_2.csv',index=False)